In [2]:
import numpy as np
import scipy
import matplotlib.pyplot as plt

In [ ]:
class Quadrature:
    def __init__(self, order):
        self.order = order
        self.mus = None
        self.weights = None
        
    def compute_quadrature(self):
        # Compute the quadrature points and weights
        # In 1D slab geometry, the quadrature points are the Legendre roots
        # and the weights are the corresponding Legendre weights
        (self.mus, self.weights) = \
            np.polynomial.legendre.leggauss(self.order)
        
        # By default, angles go from negative to positive, and we want the opposite 
        self.mus = np.flip(self.mus)
class Mesh:
    def __init__(self, max_mesh_width, num_mesh_points):
        self.max_mesh_width = max_mesh_width
        self.num_mesh_points = num_mesh_points
        self.dx = None 
        self.mesh = None
    
    def create_mesh(self):
        # Number of mesh intervals and mesh spacing
        self.mesh, self.dx = np.linspace(0, self.max_mesh_width, self.num_mesh_points, retstep=True)

class MaterialProperties:
    def __init__(self, zones_sigma_t, zones_sigma_a, zones_sigma_s, zones_q):
        self.zones_sigma_t = zones_sigma_t # Dictionary where keys are tuples zone = (zone_begin, zone_end) and values are sigma_t for that zone
        self.zones_sigma_a = zones_sigma_a # Same as above, but for sigma_a
        self.zones_sigma_s = zones_sigma_s # Same as above, but for sigma_s
        self.zones_q       = zones_q       # Same as above, but for external volumetric sources

    def return_xs(self, mesh_point):
        sigma_t = None
        sigma_a = None
        sigma_s = None

        for (zone, sigma_t_val) in self.zones_sigma_t.items():
            if zone[0] <= mesh_point <= zone[1]:
                sigma_t = sigma_t_val

        for (zone, sigma_a_val) in self.zones_sigma_a.items():
            if zone[0] <= mesh_point <= zone[1]:
                sigma_a = sigma_a_val

        for (zone, sigma_s_val) in self.zones_sigma_s.items():
            if zone[0] <= mesh_point <= zone[1]:
                sigma_s = sigma_s_val

        return (sigma_t, sigma_a, sigma_s)        
             

class Solver:
    def __init__(self, left_bc, right_bc, zones_sigma_t,
                 zones_sigma_a, zones_sigma_s, zones_q,

                 solver_type="diamond-difference"):
